# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
val housing = spark.read.format("csv")
                        .option("inferSchema", "true")
                        .option("header", "true")
                        .load("data/housing.csv")

housing = [longitude: double, latitude: double ... 8 more fields]


[longitude: double, latitude: double ... 8 more fields]

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
housing.printSchema

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [ ]:
housing.count

## 2.2. Look at the data
Print the first five records of the dataset.

In [ ]:
housing.show(5)

Print the number of records with population more than 10000.

In [ ]:
housing.where('population > 10000).count

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [ ]:
housing.describe("housing_median_age", "total_rooms","median_house_value", "population").show

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [ ]:
import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age")).show

housing.agg(min("total_rooms")).show

housing.agg(avg("median_house_value")).show

## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [ ]:
housing.groupBy("ocean_proximity").count.orderBy(desc("count")).show

Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [ ]:
housing.groupBy("ocean_proximity").agg(avg("median_house_value").alias("avg_value")).show

Rewrite the above question in SQL.

In [ ]:
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show

## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value", "population")).setOutputCol("features")

val housingAttrs = va.transform(housing)

housingAttrs.show(5)

In [ ]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [ ]:
val housingCol1 = housing.withColumn("rooms_per_household", 'total_rooms / 'households)
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", 'total_bedrooms / 'total_rooms)
val housingExtra = housingCol2.withColumn("population_per_household", 'population / 'households)

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [ ]:
val renamedHousing = housingExtra.withColumnRenamed("median_house_value", "label")
renamedHousing.printSchema

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [ ]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [ ]:
for (c <- colNum) {
    print(s"Nulls in column $c: ")
    println(renamedHousing.where(col(c).isNull).count)
}

As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [ ]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms", "bedrooms_per_room")).setOutputCols(Array("total_bedrooms", "bedrooms_per_room"))
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

for (c <- colNum) {
    print(s"Nulls in column $c: ")
    println(imputedHousing.where(col(c).isNull).count)
}

### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vector of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [ ]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("imputed_features")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("imputed_features").setOutputCol("rescaled_features")
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [ ]:
renamedHousing.select("ocean_proximity").distinct.show

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [ ]:
import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_num")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [ ]:
indexer.fit(renamedHousing).labels

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [ ]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("ocean_proximity_num")).setOutputCols(Array("ocean_proximity_oh"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer, va, scaler))
val catPipeline = new Pipeline().setStages(Array(indexer, encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [ ]:
// val newCols = Array("longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "label", "rooms_per_household", "bedrooms_per_room", "population_per_household", "imputed_features", "rescaled_features", "ocean_proximity_num", "ocean_proximity_oh")
val newCols = Array("rescaled_features", "ocean_proximity_oh")
// val newCols = Array("rescaled_features")

val va2 = new VectorAssembler().setInputCols(newCols).setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

In [ ]:
// dataset.select("features").collect().foreach(println)

---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [ ]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(80,20))

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [ ]:
import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression
// lr.setFeaturesCol("features")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: " + lrModel.coefficients + s", Intercept: " + lrModel.intercept)
println(s"RMSE: " + trainingSummary.rootMeanSquaredError)

Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [ ]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [ ]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [ ]:
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setFeaturesCol("features").setLabelCol("label")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setMetricName("rmse").setPredictionCol("prediction").setLabelCol("label")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder()
    .addGrid(rfModel.numTrees, Array(1,  5, 10))
    .addGrid(rfModel.maxDepth, Array(5, 10, 15))
    .build()

val evaluator = new RegressionEvaluator()
    .setLabelCol("label")
    .setPredictionCol("prediction")
    .setMetricName("rmse")

val cv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(dataset)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [ ]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "The first input column")
    final val inputCol2 = new Param[String](this, "inputCol2", "The second input column")
    final val outputCol = new Param[String](this, "outputCol", "The output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    // Check that the input type is a double
    val idx1 = schema.fieldIndex($(inputCol1))
    val idx2 = schema.fieldIndex($(inputCol2))
    val field1 = schema.fields(idx1)
    val field2 = schema.fields(idx2)
    if (field1.dataType != DoubleType) {
        throw new Exception(s"Input1 type ${field1.dataType} did not match input type DoubleType")
    }
    if (field2.dataType != DoubleType) {
        throw new Exception(s"Input2 type ${field2.dataType} did not match input type DoubleType")
    }
    // Add the return field
    schema.add(StructField($(outputCol), DoubleType, false))
  }
}


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [ ]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}

import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.types.StructField

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("configurablewordcount"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    }

    override def transformSchema(schema: StructType): StructType = {
        // Add the return field
        schema.add(StructField($(outputCol), DoubleType, false))
      }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        dataset.withColumn($(outputCol), col($(inputCol1)) / col($(inputCol2)))
  }
}

Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [ ]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

In [ ]:
class LR() extends Serializable {
    
    
// https://stackoverflow.com/questions/44450889/why-does-spark-shell-fail-to-load-a-file-with-class-with-rdd-imported
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
    
    
    
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

In [ ]:
abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends org.apache.spark.ml.PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: org.apache.spark.ml.linalg.Vector, val trainingError: Array[Double])
    extends MyLinearModel[org.apache.spark.ml.linalg.Vector, MyLinearModelImpl] {

  override def copy(extra: org.apache.spark.ml.param.ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: org.apache.spark.ml.linalg.Vector): Double = {
//     println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

In [ ]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(trainSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

Col `PAY_0` should be renamed to `PAY_1`.

In [1]:
val cc = spark.read.format("csv")
    .option("inferSchema", "true")
    .option("header", "true")
    .load("data/ccdefault.csv")
    .withColumnRenamed("PAY_0", "PAY_1")
    .withColumnRenamed("DEFAULT", "label")

cc.cache()

cc = [ID: int, LIMIT_BAL: int ... 23 more fields]


[ID: int, LIMIT_BAL: int ... 23 more fields]

In [2]:
cc.printSchema

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_1: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- label: integer (nullable = true)

In [53]:
cc.count

30000

In [54]:
cc.show

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    1|
|  3|

In [55]:
cc.describe("LIMIT_BAL", "AGE", "PAY_1", "BILL_AMT1", "PAY_AMT1","label").show
// cc.describe(cc.columns:_*).show

+-------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|summary|         LIMIT_BAL|              AGE|             PAY_1|        BILL_AMT1|         PAY_AMT1|              label|
+-------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|  count|             30000|            30000|             30000|            30000|            30000|              30000|
|   mean|167484.32266666667|          35.4855|           -0.0167|       51223.3309|        5663.5805|             0.2212|
| stddev|129747.66156720246|9.217904068090155|1.1238015279973335|73635.86057552966|16563.28035402577|0.41506180569093254|
|    min|             10000|               21|                -2|          -165580|                0|                  0|
|    max|           1000000|               79|                 8|           964511|           873552|                  1|
+-------+---------------

In [8]:
for (c <- cc.columns) {
    print(s"Number of unique values in column $c: ")
//     cc.select(col(c)).distinct.show
    println(cc.select(col(c)).distinct.count)
}

Name: Compile Error
Message: <console>:33: error: not found: value col
           println(cc.select(col(c)).distinct.count)
                             ^

StackTrace: 

In [9]:
for (c <- cc.columns) {
    print(s"Nulls in column $c: ")
    println(cc.where(col(c).isNull).count)
}

Name: Compile Error
Message: <console>:32: error: not found: value col
           println(cc.where(col(c).isNull).count)
                            ^

StackTrace: 

### Prepare one hot encoding

After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute ocean_proximity.

In [10]:
// import org.apache.spark.ml.feature.IntegerIndexer

// val indexer = new IntegerIndexer()
//     .setInputCol("SEX")
//     .setOutputCol("SEX_ind")

// val ohHousing = indexer.fit(cc).transform(cc)

// ohHousing.show(5)

Name: Syntax Error.
Message: 
StackTrace: 

In [11]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("AGE")).setOutputCols(Array("AGE_oh"))
val cc_oh = encoder.fit(cc).transform(cc)

cc_oh.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+---------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|         AGE_oh|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+---------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|(79,[24],[1.0])|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|    

encoder = oneHotEncoder_e5d13042dcd1
cc_oh = [ID: int, LIMIT_BAL: int ... 24 more fields]


[ID: int, LIMIT_BAL: int ... 24 more fields]

### Vector Assembler

In [56]:
import org.apache.spark.ml.feature.VectorAssembler

// feature columns
val featCols = cc.columns.filter(_ !="label").filter(_ !="ID")

println(featCols)

val va = new VectorAssembler().setInputCols(featCols).setOutputCol("features")

val ccAttrs = va.transform(cc)

ccAttrs.select("features").show(5)

[Ljava.lang.String;@314b8c37
+--------------------+
|            features|
+--------------------+
|[20000.0,2.0,2.0,...|
|[120000.0,2.0,2.0...|
|[90000.0,2.0,2.0,...|
|[50000.0,2.0,2.0,...|
|[50000.0,1.0,2.0,...|
+--------------------+
only showing top 5 rows



featCols = Array(LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, DEFAULT)
va = vecAssembler_ae9aff5e7123
ccAttrs = [ID: int, LIMIT_BAL: int ... 24 more fields]


[ID: int, LIMIT_BAL: int ... 24 more fields]

In [ ]:
// ccAttrs.select("features").collect().foreach(println)

## Models

In [ ]:
val Array(trainSet, testSet) = ccAttrs.randomSplit(Array(80,20))
// trainSet.printSchema

### Logistic Regression

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression

// train the model
val lr = new LogisticRegression
// lr.setFeaturesCol("features")
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: " + lrModel.coefficients + s", Intercept: " + lrModel.intercept)
// println(s"RMSE: " + trainingSummary.rootMeanSquaredError)

In [ ]:
val trainingSummary = lrModel.binarySummary
// obtain the objective per iteration.
val objectiveHistory = trainingSummary.objectiveHistory
// objectiveHistory.foreach(loss => println(loss))
// obtain the ROC as a dataframe and areaUnderROC.
val roc = trainingSummary.roc
roc.show()
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")
// set the model threshold to maximize F-Measure
val fMeasure = trainingSummary.fMeasureByThreshold
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where($"F-Measure" === maxFMeasure)
    .select("threshold").head().getDouble(0)
lrModel.setThreshold(bestThreshold)

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val lrParamGrid = new ParamGridBuilder()
    .addGrid(lrModel.regParam, Array(0.01, 0.1, 1))
    .build()

val lrEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val lrCv = new CrossValidator()
    .setEstimator(lr)
    .setEvaluator(lrEvaluator)
    .setEstimatorParamMaps(lrParamGrid)
    .setNumFolds(3)

val lrCvModel = lrCv.fit(ccAttrs)

val lrPredictions = lrCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val lrAuc = lrEvaluator.evaluate(lrPredictions)
println(s"Area Under ROC (AUC) on test data = $lrAuc")

### Decision Tree

In [ ]:
import org.apache.spark.ml.classification.DecisionTreeClassifier

val dt = new DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("label")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val dtPredictions = dtModel.transform(testSet)
dtPredictions.select("prediction", "rawPrediction", "label", "features").show(5)
// predictions.printSchema

In [ ]:
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val dtPredictionAndLabels = dtPredictions.select("rawPrediction", "label").rdd.map(x => (x(0).asInstanceOf[DenseVector](1), x(1).toString.toDouble))
val dtMetrics = new BinaryClassificationMetrics(dtPredictionAndLabels)

dtMetrics.areaUnderROC

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

// DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
//             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
//             min_samples_split=42, min_weight_fraction_leaf=0.0,
//             presort=False, random_state=0, splitter='best')
// # Use a grid over parameters of interest
// param_grid = { 
//            "n_estimators" : [9, 18, 27, 36, 45, 54, 63],
//            "max_depth" : [1, 5, 10, 15, 20, 25, 30],
//            "min_samples_leaf" : [1, 2, 4, 6, 8, 10]}

val dtParamGrid = new ParamGridBuilder()
    .addGrid(dtModel.maxDepth, Array(1, 2, 4, 8, 16, 30))
    .addGrid(dtModel.maxBins, Array(2, 4, 8, 16, 32))
    .build()

val dtEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val dtCv = new CrossValidator()
    .setEstimator(dt)
    .setEvaluator(dtEvaluator)
    .setEstimatorParamMaps(dtParamGrid)
    .setNumFolds(3)

val dtCvModel = dtCv.fit(ccAttrs)

val dtPredictions = dtCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val dtAuc = dtEvaluator.evaluate(dtPredictions)
println(s"Area Under ROC (AUC) on test data = $dtAuc")

### Random Forest

In [ ]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}

val rf = new RandomForestClassifier().setFeaturesCol("features").setLabelCol("label")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val rfPredictions = rfModel.transform(testSet)
rfPredictions.select("prediction", "label", "features").show(5)

In [ ]:
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

val rfPredictionAndLabels = rfPredictions.select("rawPrediction", "label").rdd.map(x => (x(0).asInstanceOf[DenseVector](1), x(1).toString.toDouble))
val rfMetrics = new BinaryClassificationMetrics(rfPredictionAndLabels)

rfMetrics.areaUnderROC

In [84]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.tuning.CrossValidator

// val rfParamGrid = new ParamGridBuilder()
//     .addGrid(rfModel.maxDepth, Array(1, 2, 4, 8, 16, 30))
//     .addGrid(rfModel.maxBins, Array(2, 4, 8, 16, 32))
//     .build()

val rfParamGrid = new ParamGridBuilder()
//     .addGrid(rfModel.maxDepth, Array(1, 2, 4, 8))
//     .addGrid(rfModel.maxBins, Array(2, 4, 8))
//     .addGrid(rfModel.minInfoGain, Array(0.001, 0.01, 0.1))
    .addGrid(rfModel.maxDepth, Array(1, 2, 4, 8, 16, 30))
    .addGrid(rfModel.maxBins, Array(2, 4, 8, 16))
    .addGrid(rfModel.minInfoGain, Array(0.001, 0.01))
    .build()

val rfEvaluator = new BinaryClassificationEvaluator()
    .setLabelCol("label")

val rfCv = new CrossValidator()
    .setEstimator(rf)
    .setEvaluator(rfEvaluator)
    .setEstimatorParamMaps(rfParamGrid)
    .setNumFolds(3)

val rfCvModel = rfCv.fit(ccAttrs)

val rfPredictions = rfCvModel.transform(testSet)
lrPredictions.select("prediction", "label", "features").show(5)

val rfAuc = rfEvaluator.evaluate(rfPredictions)
println(s"Area Under ROC (AUC) on test data = $rfAuc")

Name: Compile Error
Message: <console>:4: error: error writing object $eval: /tmp/spark-7bf2a6f3-cd2a-4d29-bd51-7852ba51e738/repl-417269e0-1fc5-4107-b72d-855600ba1f66/class: /tmp/spark-7bf2a6f3-cd2a-4d29-bd51-7852ba51e738/repl-417269e0-1fc5-4107-b72d-855600ba1f66/$line328 is not a directory
object $eval {
       ^
org.apache.spark.SparkException: Exception thrown in awaitResult:
  at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:205)
  at org.apache.spark.ml.tuning.CrossValidator$$anonfun$4$$anonfun$6.apply(CrossValidator.scala:164)
  at org.apache.spark.ml.tuning.CrossValidator$$anonfun$4$$anonfun$6.apply(CrossValidator.scala:164)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
  at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
  at scala

## From reference

Reference:
https://hub.packtpub.com/how-to-win-kaggle-competition-with-apache-sparkml/

In [8]:
import org.apache.spark.ml.feature.OneHotEncoder
import org.apache.spark.ml.feature.{OneHotEncoderEstimator, OneHotEncoderModel}

// val indexColumns = Array("SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6")
val indexColumns = Array("SEX", "EDUCATION", "MARRIAGE", "AGE")

val encoder = new OneHotEncoderEstimator()
  .setInputCols(indexColumns)
  .setOutputCols(indexColumns map (name => s"${name}_vec"))

val cc_enc = encoder.fit(cc).transform(cc)

indexColumns = Array(SEX, EDUCATION, MARRIAGE, AGE)
encoder = oneHotEncoder_59df18d1bf9f
cc_enc = [ID: int, LIMIT_BAL: int ... 27 more fields]


lastException: Throwable = null


[ID: int, LIMIT_BAL: int ... 27 more fields]

In [9]:
import org.apache.spark.ml.feature.VectorAssembler 
import org.apache.spark.ml.linalg.Vectors

val featureColumns = Array("SEX_vec", "EDUCATION_vec", "MARRIAGE_vec", "AGE_vec", "LIMIT_BAL", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6")
// val featureColumns = cc.columns.filter(_ !="label").filter(_ !="ID")

var va = new VectorAssembler()
    .setInputCols(featureColumns)
    .setOutputCol("features")

val cc_feat = va.transform(encoder.fit(cc).transform(cc))

featureColumns = Array(SEX_vec, EDUCATION_vec, MARRIAGE_vec, AGE_vec, LIMIT_BAL, PAY_1, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
va = vecAssembler_e63c9bab2bfa
cc_feat = [ID: int, LIMIT_BAL: int ... 28 more fields]


[ID: int, LIMIT_BAL: int ... 28 more fields]

In [10]:
import org.apache.spark.ml.feature.Normalizer
import org.apache.spark.ml.linalg.Vectors

// Normalize each Vector using $L^1$ norm.
val normalizer = new Normalizer()
  .setInputCol("features")
  .setOutputCol("normFeatures")
  .setP(1.0)

val l1NormData = normalizer.transform(cc_feat)
println("Normalized using L^1 norm")
l1NormData.show()

// Normalize each Vector using $L^\infty$ norm.
val lInfNormData = normalizer.transform(cc_feat, normalizer.p -> Double.PositiveInfinity)
println("Normalized using L^inf norm")
lInfNormData.show()

Normalized using L^1 norm
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+---------------+--------------------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|      SEX_vec|EDUCATION_vec| MARRIAGE_vec|        AGE_vec|            features|        normFeatures|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+---------------+--------------------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2

normalizer = normalizer_bc331ff36798
l1NormData = [ID: int, LIMIT_BAL: int ... 29 more fields]
lInfNormData = [ID: int, LIMIT_BAL: int ... 29 more fields]


[ID: int, LIMIT_BAL: int ... 29 more fields]

In [11]:
import org.apache.spark.ml.{Pipeline, PipelineModel}

//Create an array out of individual pipeline stages 
var transformers = Array(encoder,va)
// var transformers = Array(va,normalizer)

var pipeline = new Pipeline().setStages(transformers).fit(cc)

var transformed = pipeline.transform(cc)
//     .withColumnRenamed("DEFAULT", "label")
//     .select("features", "label")

transformers = Array(oneHotEncoder_59df18d1bf9f, vecAssembler_e63c9bab2bfa)
pipeline = pipeline_1b75c087c759
transformed = [ID: int, LIMIT_BAL: int .....


[ID: int, LIMIT_BAL: int .....

In [12]:
import org.apache.spark.ml.classification.RandomForestClassifier 

var rf = new RandomForestClassifier()
    .setLabelCol("label")
    .setFeaturesCol("features")

var model = new Pipeline().setStages(transformers :+ rf).fit(cc) 
var result = model.transform(cc)

rf = rfc_602cef8b9a87
model = pipeline_193a2beefc4b
result = [ID: int, LIMIT_BAL: int ... 31 more fields]


[ID: int, LIMIT_BAL: int ... 31 more fields]

In [13]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator 

val evaluator = new BinaryClassificationEvaluator()

import org.apache.spark.ml.param.ParamMap

var evaluatorParamMap = ParamMap(evaluator.metricName -> "areaUnderROC") 
var aucTraining = evaluator.evaluate(result, evaluatorParamMap)

evaluator = binEval_3a1cd8ddb717
evaluatorParamMap = 
aucTraining = 0.7781706675050388


{
	binEval_3a1cd8ddb717-metricName: areaUnderROC
}


0.7781706675050388

In [ ]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder} 

var paramGrid = new ParamGridBuilder()
    .addGrid(rf.numTrees, 3 :: 5 :: 10 :: 30 :: 50 :: 70 :: 100 :: 150 :: Nil)
    .addGrid(rf.featureSubsetStrategy, "auto" :: "all" :: "sqrt" :: "log2" :: "onethird" :: Nil)
    .addGrid(rf.impurity, "gini" :: "entropy" :: Nil)
    .addGrid(rf.maxBins, 2 :: 5 :: 10 :: 15 :: 20 :: 25 :: 30 :: Nil)
    .addGrid(rf.maxDepth, 3 :: 5 :: 10 :: 15 :: 20 :: 25 :: 30 :: Nil)
    .build()

var crossValidator = new CrossValidator()
    .setEstimator(new Pipeline().setStages(transformers :+ rf))
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(5)
    .setEvaluator(evaluator)

var crossValidatorModel = crossValidator.fit(cc)

var newPredictions = crossValidatorModel.transform(cc)

Exception in thread "Executor task launch worker for task 7295" java.lang.SecurityException: Not allowed to invoke System.exit!
	at org.apache.toree.security.KernelSecurityManager.checkExit(KernelSecurityManager.scala:147)
	at java.lang.Runtime.halt(Runtime.java:273)
	at org.apache.spark.util.SparkUncaughtExceptionHandler.uncaughtException(SparkUncaughtExceptionHandler.scala:32)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:557)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Exception in thread "dag-scheduler-event-loop" java.lang.OutOfMemoryError: Java heap space


In [ ]:
var bestPipelineModel = crossValidatorModel.bestModel.asInstanceOf[PipelineModel]

var stages = bestPipelineModel.stages

import org.apache.spark.ml.classification.RandomForestClassificationModel 

val rfStage = stages(stages.length-1).asInstanceOf[RandomForestClassificationModel] 

rfStage.getNumTrees
rfStage.getFeatureSubsetStrategy
rfStage.getImpurity
rfStage.getMaxBins
rfStage.getMaxDepth